In [1]:
!pip install scipy
!pip install sentence_transformers

In [14]:
import torch
import torch.nn as nn
# import math
# from scipy.stats import special_ortho_group
# import matplotlib.pyplot as plt
# import random

# # for AE and Embedder
# import torch.nn.functional as F
# from sentence_transformers import SentenceTransformer
# from torch.utils.data import DataLoader, TensorDataset


# from torchvision.datasets import MNIST
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader, random_split
from cINN import ConditionalRealNVP

ae_path = "/models/ae_100.pth"

In [19]:
def train_cinn(model, x_data, y_data, batch_size=32, epochs=1000, lr=0.001):
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

    # Convert each element in x_data and y_data to a scalar
    x_data = [torch.Tensor(data_point).squeeze() for data_point in x_data]
    y_data = [torch.Tensor(data_point).squeeze() for data_point in y_data]

    data = TensorDataset(torch.stack(x_data), torch.stack(y_data))
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        for x_batch, y_batch in dataloader:
            optimizer.zero_grad()

            z, ljd = model(x_batch, y_batch)

            loss = torch.sum(0.5 * torch.sum(z**2, -1) - ljd) / batch_size
            loss.backward()

            optimizer.step()

        if (epoch+1) % (epochs//5) == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Create the training Data derived only form the Embeddig model an AE

In [5]:
emb_model = SentenceTransformer("intfloat/multilingual-e5-small")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:



AE = AutoencoderSimpler()
AE.load_state_dict(torch.load(ae_path, map_location=get_best_device()))

<All keys matched successfully>

In [7]:
import torch
from torchvision import datasets, transforms
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader

# Download MNIST dataset
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=mnist_transform)

# Create DataLoader for batch processing
data_loader = DataLoader(mnist_dataset, batch_size=64, shuffle=True)

# Assuming you have a PyTorch model named ae and an embedding model named emb_model
# ae.encoder(x) should encode an image x
# emb_model(string) should embed a label

# Replace images with encoded latent space representation and replace labels with embeddings
encoded_dataset = []

for batch_imgs, batch_labels in data_loader:
    encoded_imgs = AE.encoder(batch_imgs)
    encoded_dataset.extend(zip(encoded_imgs, batch_labels))

# Convert labels to strings if not already
encoded_dataset = [(img, str(label)) for img, label in encoded_dataset]

# Embed labels efficiently using SentenceTransformer
embedded_labels = emb_model.encode(list(zip(*encoded_dataset))[1])
embedded_dataset = list(zip(list(zip(*encoded_dataset))[0], embedded_labels))

# You can use embedded_dataset for your further processing


In [8]:
print(len(embedded_dataset))
print(len(embedded_dataset[0]))

60000
2


In [9]:
print(len(emb_model.encode("0")))

384


In [10]:
print(len(embedded_dataset[0][0]))

64


In [11]:
print(len(embedded_dataset[0][1]))

384


In [12]:
cINN = conditional_realNVP(64,128,20,384)

In [13]:
encoded_imgs, embedded_labels = zip(*embedded_dataset)

In [20]:
train_cinn(cINN, encoded_imgs, embedded_labels, batch_size=128)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3